**Bert model op basis van Bert van Universiteit van Groningen. Context handling moet nog aangepast worden. Oversampling a.d.h.v de mediaan. dynamisch treshhold zoeken voor unknown.**

In [1]:
import pandas as pd
import re
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
import torch.nn.functional as F
import os 

c:\Users\corne\anaconda3\envs\thesis-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ✅ 1. Load & Preprocess Data
script_dir = os.path.dirname(os.getcwd()) # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_NoDupsLessThemesENnoWords9.xlsx")
df = pd.read_excel(file_path)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64572 entries, 0 to 64571
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   context      64262 non-null  object
 1   question     64572 non-null  object
 2   statistical  64572 non-null  int64 
 3   theme        64572 non-null  object
 4   file_name    64572 non-null  object
 5   clean_text   64572 non-null  object
dtypes: int64(1), object(5)
memory usage: 3.0+ MB


,context,question,statistical,theme,file_name,clean_text
0,Ondertussen is de eerstelijnszone BruZEL al me...,Zoals alle eerstelijnszones kreeg ook BruZEL h...,0,Brussel en de Vlaamse Rand,1752898.txt,Zoals alle eerstelijnszones kreeg ook BruZEL h...
1,Ondertussen is de eerstelijnszone BruZEL al me...,2.Kan de minister toelichten op welke manier B...,0,Brussel en de Vlaamse Rand,1752898.txt,Kan de minister toelichten op welke manier Bru...
2,Ondertussen is de eerstelijnszone BruZEL al me...,3.Kan de minister in het bijzonder toelichten ...,0,Brussel en de Vlaamse Rand,1752898.txt,Kan de minister in het bijzonder toelichten op...
3,Ondertussen is de eerstelijnszone BruZEL al me...,4.Kan de minister in het bijzonder toelichten ...,0,Brussel en de Vlaamse Rand,1752898.txt,Kan de minister in het bijzonder toelichten op...
4,Ondertussen is de eerstelijnszone BruZEL al me...,Zoals alle eerstelijnszones kreeg ook BruZEL h...,0,Brussel en de Vlaamse Rand,1752906.txt,Zoals alle eerstelijnszones kreeg ook BruZEL h...


In [3]:
import re

# Drop unnecessary columns
columns_to_drop = ["context","file_name","question","statistical"]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# ✅ Drop rare themes (appearing < 2 times)
theme_counts = df["theme"].value_counts()
valid_themes = theme_counts[theme_counts >= 100].index
df = df[df["theme"].isin(valid_themes)]


# ✅ Recompute label encoding AFTER filtering
unique_themes = list(df["theme"].unique())
theme_to_id = {theme: idx for idx, theme in enumerate(unique_themes)}
id_to_theme = {idx: theme for theme, idx in theme_to_id.items()}
df["theme_id"] = df["theme"].map(theme_to_id)

In [4]:

# 1. First split into train + temp
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df["clean_text"].tolist(), df["theme_id"].tolist(),
    test_size=0.3, stratify=df["theme_id"], random_state=42
)

# 2. Then split temp into validation + test (e.g., 15% val, 15% test)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels,
    test_size=0.5, stratify=temp_labels, random_state=42
)


In [5]:
import pandas as pd

# Create a small DataFrame from train lists
train_df = pd.DataFrame({
    "clean_text": train_texts,
    "theme_id": train_labels
})

# Compute class counts and use median as balancing target
theme_counts = train_df["theme_id"].value_counts()



from collections import Counter
print("Class distribution :", Counter(train_labels))


Class distribution : Counter({9: 12014, 2: 7021, 13: 5398, 10: 4649, 4: 3636, 6: 2900, 8: 1628, 12: 1603, 1: 1548, 14: 979, 3: 754, 7: 603, 11: 599, 16: 548, 15: 531, 0: 392, 5: 284, 17: 113})


In [6]:
# ✅ 7. Load BERT Tokenizer & Define Dataset Class
from transformers import RobertaTokenizer, RobertaForSequenceClassification

model_name = "pdelobelle/robbert-v2-dutch-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)

class ThemeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx], padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt"
        )
        encoding["labels"] = torch.tensor(self.labels[idx])
        return {key: val.squeeze(0) for key, val in encoding.items()}

train_dataset = ThemeDataset(train_texts, train_labels, tokenizer)
validation_dataset = ThemeDataset(val_texts, val_labels, tokenizer)
test_dataset = ThemeDataset(test_texts, test_labels, tokenizer)

In [7]:
# ✅ 8. Load BERT Model for Classification
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=df["theme_id"].nunique())

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from sklearn.utils.class_weight import compute_class_weight
import torch
import numpy as np

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

# ✅ Clip only the upper bound at 10
clipped_weights = np.clip(class_weights, None, 10.0)

# Convert to PyTorch tensor
class_weights_tensor = torch.tensor(clipped_weights, dtype=torch.float)


In [9]:
from transformers import Trainer
from torch.nn import CrossEntropyLoss

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = CrossEntropyLoss(weight=class_weights_tensor.to(model.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [10]:
# ✅ 9. Define Training Arguments (With Early Stopping)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",               # 👈 No auto-checkpoints
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,  
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",  # 🔥 Log only once per epoch
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    warmup_ratio=0.1,  # 🔥 10% of training for warmup 
)

# ✅ 10. Define Metrics for Evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=1)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ✅ 11. Train Model with Early Stopping
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop if no improvement for 2 epochs
)

trainer.train()

c:\Users\corne\anaconda3\envs\thesis-env\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 12%|█▎        | 5650/45200 [22:31<2:46:05,  3.97it/s]

{'loss': 2.082, 'grad_norm': 13.524598121643066, 'learning_rate': 1.9446411012782697e-05, 'epoch': 1.0}


                                                      
 12%|█▎        | 5650/45200 [23:44<2:46:05,  3.97it/s]

{'eval_loss': 1.5063101053237915, 'eval_accuracy': 0.6450547181499071, 'eval_precision': 0.663977968199459, 'eval_recall': 0.6450547181499071, 'eval_f1': 0.6450498258659496, 'eval_runtime': 72.1737, 'eval_samples_per_second': 134.204, 'eval_steps_per_second': 16.779, 'epoch': 1.0}


 25%|██▌       | 11300/45200 [44:45<1:56:59,  4.83it/s] 

...

                                                       
100%|██████████| 45200/45200 [2:55:59<00:00,  4.81it/s]

{'eval_loss': 1.9492195844650269, 'eval_accuracy': 0.7694610778443114, 'eval_precision': 0.767899883068991, 'eval_recall': 0.7694610778443114, 'eval_f1': 0.7681818612920336, 'eval_runtime': 54.885, 'eval_samples_per_second': 176.478, 'eval_steps_per_second': 22.064, 'epoch': 8.0}


100%|██████████| 45200/45200 [2:56:01<00:00,  4.28it/s]

{'train_runtime': 10561.0795, 'train_samples_per_second': 34.239, 'train_steps_per_second': 4.28, 'train_loss': 0.7542190659784638, 'epoch': 8.0}


TrainOutput(global_step=45200, training_loss=0.7542190659784638, metrics={'train_runtime': 10561.0795, 'train_samples_per_second': 34.239, 'train_steps_per_second': 4.28, 'total_flos': 9.51546253344768e+16, 'train_loss': 0.7542190659784638, 'epoch': 8.0})

SAVE MODEL NAAR KUL DRIVE 

In [11]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import json


# === Create timestamped save path in OneDrive ===
run_id = datetime.now().strftime("%Y-%m-%d_%H-%M")
base_path = "C:/Users/corne/OneDrive - KU Leuven/Thesis/Working Code/SAVED-Models/final/roBERT"
save_path = os.path.join(base_path, f"Run_{run_id}")
os.makedirs(save_path, exist_ok=True)

# === Save model using Trainer ===
trainer.save_model(save_path)

# === Get predictions ===
preds_output = trainer.predict(test_dataset)
logits = preds_output.predictions
predictions = np.argmax(logits, axis=1)

# === Save predictions to CSV ===
output_df = pd.DataFrame({
    "text": test_texts,  # make sure test_texts is defined
    "true_label": test_labels,  # make sure test_labels is defined
    "predicted_label": predictions,
    "logits": logits.tolist()
})

csv_path = os.path.join(save_path, "test_predictions.csv")
output_df.to_csv(csv_path, index=False)

# === Optional: Save a description file ===
description = """
Model: GroNLP BERT-based Dutch Cased
Training Details:
geen context meegegeven in zowel train als test
8 epochs 

oversampling via jef zijn weights manier

data set : Grote_data_NoDupsLessThemesENnoWords9

Train test en validation set!! 
"""

desc_path = os.path.join(save_path, "model_description.txt")
with open(desc_path, "w", encoding="utf-8") as f:
    f.write(description)

# === Save label mappings ===
mappings_path = os.path.join(save_path, "label_mappings.json")
with open(mappings_path, "w", encoding="utf-8") as f:
    json.dump({
        "theme_to_id": theme_to_id,
        "id_to_theme": {str(k): v for k, v in id_to_theme.items()},  # keys must be str for JSON
        "unique_themes": unique_themes  # Save the unique themes list

    }, f, ensure_ascii=False, indent=4)


print(f"Everything saved in: {save_path}")


100%|██████████| 1211/1211 [00:55<00:00, 21.68it/s]


Everything saved in: C:/Users/corne/OneDrive - KU Leuven/Thesis/Working Code/SAVED-Models/final/roBERT\Run_2025-05-04_22-35
